In [16]:
%pip install fastapi pydantic toml

Note: you may need to restart the kernel to use updated packages.


In [17]:
# from rs import skill_check

# def test_skill_check(metadata):
#     print(skill_check("sanity", None, metadata))

# metadata = {
#     "character": {"sanity":30},
#     "symptom":[
#         "incoherent speech",
#         "uncontrollable twitching, trembling",
#         "delusions of persecution",
#         "strange appetites (dirt, clay, etc...)",
#         "scratches, punches, and bruises all over the body"
#     ]
# }

# test_skill_check(metadata)
# test_skill_check(metadata)
# test_skill_check(metadata)
# test_skill_check(metadata)
# test_skill_check(metadata)
# test_skill_check(metadata)
# test_skill_check(metadata)
# test_skill_check(metadata)
# test_skill_check(metadata)
# test_skill_check(metadata)
# test_skill_check(metadata)
# test_skill_check(metadata)

# print(metadata["character"])


In [18]:
import re
import tomllib
import functools
from typing import Literal

import ipywidgets as widgets
from IPython.display import display

from dotenv import load_dotenv

load_dotenv()

from rs import Message, Config, chat, skill_check
import d20

action_pattern = re.compile(r"(\d+). *(.+)")
skill_pattern = re.compile(r"[\w_]+")
skill_difficulty_pattern = re.compile(r"(easy|normal|hard)")

def next_round(b):
    global current_round, initial_messages
    current_round += 1
    initial_messages = config.initial_messages(current_round, config.metadata)

    print(initial_messages[-1].content)
    display(get_controls(initial_messages[-1]))

def get_user_message_tail():
    count = 0
    for msg in initial_messages:
        if msg.role == "user":
            count += 1

    is_final =  ", FINAL ROUND" if  current_round == len(config.rounds) else ""
    
    # print(config.metadata["character"])
    
    if "tags" in config.metadata["character"] and "insane" in config.metadata["character"]["tags"]:
        return f"TURN: {count + 1} {is_final} (I'm insane!)"
    
    return f"TURN: {count + 1} {is_final}"
    

def do_chat(role:Literal["user", "system", "assistant"], content: str):
    print(f"{content}\n\n")

    initial_messages.append(Message(role=role, content=content))
    msg = chat(initial_messages, lambda delta: print(delta, end=""))

    initial_messages.append(msg)
    display(get_controls(msg))


def on_skill(b, skill:str, difficulty:str):
    # print(config.metadata["character"])
    content = skill_check(skill, difficulty, config.metadata)
    content += f"\n{get_user_message_tail()}"
    do_chat("user", content)

def on_action(b, index:int):
    content = f"I select option {index}. {get_user_message_tail()}"
    do_chat("user", content)

def on_custom_input(b, input:widgets.Text):
    content = f"{input.value}. {get_user_message_tail()}"
    do_chat("user", content)

def on_custom_action(b):
    hbox = widgets.HBox(layout=widgets.Layout(width="100%"))
    action_text = widgets.Text(layout=widgets.Layout(width="100%"))
    submit = widgets.Button(description="SUBMIT", layout=widgets.Layout(width="fit-content"))
    submit.on_click(functools.partial(on_custom_input, input=action_text))
    hbox.children += (action_text, submit) 
    display(hbox)

def get_controls(msg:Message) -> widgets.Box:
    dict = msg.dict()
    buttons = []
    if "possible actions" in dict.keys():
        matches = action_pattern.finditer(dict["possible actions"])
        for match in matches:
            btn = widgets.Button(description=match.group(1), tooltip=match.group(2), layout=widgets.Layout(width="auto"))
            btn.on_click(functools.partial(on_action, index=int(match.group(1))))
            buttons.append(btn)
    elif "skill" in dict.keys() and "difficulty" in dict.keys():
        match = skill_pattern.match(dict["skill"])
        skill = match.group(0)
        match = skill_difficulty_pattern.match(dict["difficulty"])
        difficulty = match.group(0)
        skill_button = widgets.Button(description=f"{skill.upper()} [{difficulty.upper()}]", layout=widgets.Layout(width="auto"))
        skill_button.on_click(functools.partial(on_skill, skill=skill, difficulty=difficulty))
        buttons.append(skill_button)
    elif "summary" in dict.keys():
        # add summary as a memory to the metadata
        # it will be used in the next round.
        # config.metadata["memories"] = []
        config.metadata["memories"].append(dict["summary"])
        next_btn = widgets.Button(description="NEXT ROUND", layout=widgets.Layout(width="auto"))
        next_btn.on_click(next_round)
        buttons.append(next_btn)
    
    custom_action_button = widgets.Button(description="CUSTOM")
    custom_action_button.on_click(on_custom_action)
    buttons.append(custom_action_button)

    return widgets.HBox(buttons)

# open config file
config_name = "data/coc_lot_3636.toml"
config: Config | None = None
with open(config_name, "rb") as f:
    data = tomllib.load(f)
    config = Config(**data)

current_round = 4
initial_messages = config.initial_messages(current_round, config.metadata)

print(initial_messages[-1].content)
display(get_controls(initial_messages[-1]))



Professor Dagger examines the seance table with utmost concentration and scrutiny, being as careful as possible to not inflict any damage to the table. He then shifted his attention to the three leather bound books, inspecting its condition while flipping through its contents to verify its authenticity. 

After much inspection, he turned his attention towards you, and inquired about a fourth book, expressing that altogether, it could fetch a price well above $2,000,000. Your eyes lit up, recognizing this to be your chance to turn your life around, pay off your debts and perhaps start something by yourself. The mystery of the fourth book piqued your interest despite the disinterest in the occult, and such curiosity beckoned you to ask about the importance of the fourth book. A heavy silence ensued, the Professor’s shadowy face under the chandelier’s soft  glow gazed into your eyes with great earnestness. In a somber tone, he explained that the fourth book is the rarest of them all, fo

I select option 1. TURN: 1 


As the truck hums down the road, you nod to Professor Dagger, signaling your willingness to listen. He clears his throat, the weight of his words seeming to press down on the air between you. "Kyle," he begins, his voice tinged with a seriousness that commands your full attention, "the previous owner of the storage you now possess was deeply entangled in the occult. Not just him, but his entire family. They were known to dabble in practices that many would consider...abominable."

He pauses, as if gauging your reaction, before continuing, "Among their dark endeavors was a ritual, performed with the very book we seek. It's said that they summoned a demon and bound it to possess one of their own, the sister of the deceased owner. The events that followed are shrouded in whispers, legends, and unspeakable horrors. Many believe that the knowledge and artifacts contained within that storage could be of great power...or a terrible curse."

The revelation sends a


Memories:
"""
- Kyle chose to listen to Professor Dagger's confession about the previous owner's involvement in the occult and the dark history surrounding the storage unit. They have arrived at the self-storage facility, ready to uncover the mysteries that lie within.
- Kyle Jackson, driven by financial desperation, rejected a middle-aged woman's request to access the storage room he had recently purchased, hoping to find valuable items to sell. Upon entering the storage, Kyle discovered strange and unsettling objects, including a collection of old jars with unidentifiable contents, which he failed to appraise. His investigation led him to a hidden compartment containing maritime maps, a rusted compass, and a journal detailing a voyage to the mythical R'lyeh, driving him to the brink of madness. As Kyle loses control, haunted by the implications of his findings, he hears a strange noise, indicating that something else is present in the storage room with him.
"""


You invited Profess

I select option 1. TURN: 1 


You decide to follow Professor Dagger closely in silence, your footsteps echoing softly in the narrow, dimly lit corridor. The air grows colder as you proceed, the walls seeming to close in around you. The pungent odor intensifies, making it difficult to keep a steady breath. You notice Professor Dagger's pace quicken slightly, a sense of urgency in his movements. You match his pace, keeping your focus ahead, trying to ignore the creeping dread that seems to fill the corridor.

As you reach the end of the corridor, the sight before you halts you in your tracks. There lies a near-withered body of a woman, sprawled across the floor over a pentagram made of a strange red fluid. The face of the body is completely gone, leaving a gaping cavity where something sinister writhes within. Despite Professor Dagger's earlier warning, the grotesque scene before you demands your attention, and a chill runs down your spine.

You need to make a skill check to maintain you

I am making a SANITY check against my own sanity points: 10.
And... FAILED! I lost 3 sanity points.
TURN: 2 


As the shock of the grotesque scene overwhelms you, your mind reels from the horror of the withered body and the sinister presence writhing within its facial cavity. You struggle to maintain your composure, but the sight is too much for your psyche to bear. You feel a part of your sanity slip away as you lose 3 sanity points, leaving you with 7 sanity points.

The realization hits you hard, and for a moment, you feel the urge to flee, to escape this nightmare that has become all too real. However, Professor Dagger, seemingly unfazed by the sight, gestures for you to follow him further into the room, towards a podium at the far end. On it rests the fourth book, its cover adorned with symbols that seem to pulsate under the flickering light.

Despite your shaken state, you understand the importance of this discovery. The book may hold the key to not only understanding the dark hi

I select option 1. TURN: 3 


Gathering the remnants of your courage, you approach the podium to examine the book. Each step towards it feels heavy, as if the air around you thickens with anticipation. The book lies open, its pages filled with arcane symbols and writings in a language you cannot comprehend, yet you feel a strange pull towards it, a compulsion to understand its secrets.

As you reach out to touch the book, the strange red fluid forming the pentagram beneath the withered body begins to evaporate, disappearing without a trace. This sudden change causes you to hesitate, but your curiosity drives you to grasp the book firmly, ready to remove it from the podium.

The moment the book is dislodged, an unholy shriek pierces the air. You watch in horror as tentacles begin pouring out from the withered body's facial cavity, writhing and thrashing with a life of their own. They latch onto Professor Dagger, who is too close to the body, and before your very eyes, he is consumed by 

I select option 1. TURN: 4 


With adrenaline surging through your veins, you turn on your heels and sprint towards the corridor, the only exit you know of. The chilling sound of the creature's movements echoes behind you, a constant reminder of the imminent danger nipping at your heels. You dare not look back, focusing solely on the path ahead, your breath ragged as you push your body to its limits.

The dim light of the corridor grows brighter as you approach the entrance, the promise of escape fueling your desperate run. You recall the facility's layout in your mind, the twists and turns you took to get here, and the locked exit door that stands between you and freedom.

As you burst into the open space of the storage facility, the realization hits you – the exit door. The memory of the middle-aged woman you had turned away flashes through your mind. Had she, in a moment of gratitude, left the door unlocked for you?

You reach the door, your hands frantically searching for a way to 

stand your ground and fight. TURN: 5 


In a moment of desperation and defiance, you decide to stand your ground and fight. Knowing full well the creature's capability to kill in one hit, you understand this might be your last stand. Your mind races, searching for any advantage or weapon within your immediate surroundings that could give you a fighting chance against this unearthly foe.

You recall the items you've seen scattered around the storage facility - old furniture, heavy objects, and tools. Your eyes quickly settle on a hefty crowbar lying not too far from you, discarded amidst the chaos of your earlier search. With no time to waste, you dash towards it, gripping it tightly in your hands as you turn to face the oncoming horror.

The creature, a mass of tentacles and malevolence, barrels towards you with an unnatural speed. You brace yourself, crowbar raised, ready to strike with all the strength and determination you can muster. The fight for your life is about to begin.

You 

I am making a skill check using FIGHTING against a difficulty of HARD.
And I rolled a 85 for a result of FAILURE.
TURN: 6 


In the tense moments that follow, you swing the crowbar with all your might, aiming for the writhing mass of tentacles that now defines the creature's horrifying visage. However, the creature's movements are too swift, too erratic for your strike to land with any meaningful impact. Your attack passes through the air, missing the creature by mere inches as it continues its relentless advance.

The failure of your attempt leaves you exposed, the realization of your impending doom settling in as the creature closes the distance between you with terrifying speed. In the final moments, as the creature looms over you, a surge of adrenaline and instinct propels you to make one last desperate move to evade its lethal grasp.

You throw yourself to the side, narrowly avoiding the creature's strike, but the knowledge that nothing can truly stop it weighs heavily on your min

go all in and fight it again with all my might. TURN: 7 


Despite the overwhelming odds and the clear indication that fighting this creature might be a futile effort, your resolve hardens. You refuse to give in to despair, choosing instead to face this nightmare head-on. With the crowbar still in hand, you turn to face the creature, your determination unwavering. This moment, this decision, feels like the culmination of every challenge you've faced, every demon you've fought, both within and without.

You wait for the creature to make its move, timing your next attack with precision born of desperation. As it lunges towards you, you swing the crowbar with all the force you can muster, aiming for what you perceive to be its most vulnerable point - the writhing mass emerging from its facial cavity.

This time, your weapon connects, the crowbar sinking into the grotesque flesh with a sickening squelch. However, the creature's reaction is not what you expect. Instead of recoiling or showi